In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [2]:
brupt = pd.read_csv("C:/Training/Academy/Statistics (Python)/Cases/Bankruptcy/Bankruptcy.csv",
                   index_col=0)
brupt.head(3)

,D,YR,R1,R2,R3,R4,R5,R6,R7,R8,...,R15,R16,R17,R18,R19,R20,R21,R22,R23,R24
NO,,,,,,,,,,,,,,,,,,,,,
1,0,78,0.23,0.08,0.02,0.03,0.46,0.12,0.19,10.36,...,0.05,0.57,0.15,0.23,3.56,0.26,1.55,0.43,0.11,0.17
2,0,77,0.19,0.07,0.09,0.12,0.02,0.02,0.03,3.13,...,0.09,0.12,0.16,0.22,3.78,1.29,1.40,0.06,0.07,0.10
3,0,72,0.07,0.02,0.03,0.05,0.06,0.10,0.14,2.41,...,-0.03,0.02,0.02,0.04,13.29,1.61,1.43,0.03,0.05,0.07


In [3]:
X = brupt.drop(['D','YR'], axis=1).values
y = brupt['D'].values
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=24,stratify=y)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [4]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([92, 24])
torch.Size([92])


In [5]:
X_scl_trn.shape[1]

24

In [6]:
torch.manual_seed(24)
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=15),
                      nn.ReLU(),
                      nn.Linear(15, 7),
                      nn.ReLU(),
                      nn.Linear(7,1))

In [7]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [8]:
y_torch.size()

torch.Size([92])

In [9]:
y_pred = model(X_torch.float())
y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([92, 1])
torch.Size([92, 1])


### Initial Log Loss

In [10]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(0.7081, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [11]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [12]:
torch.manual_seed(24)
joint_dataset = TensorDataset(X_torch.float(), y_torch.float())
data_loader = DataLoader(dataset=joint_dataset, batch_size=16, shuffle=True)


### Training Loop

In [13]:

for epoch in np.arange(0,100):
   for i, batch in enumerate(data_loader, 1):
      # Forward pass: Compute predicted y by passing x to the model
      y_pred_prob = model(batch[0].float())

      # Compute and print loss
      loss = criterion(y_pred_prob, batch[1].view(-1,1).float())
      

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()

      # perform a backward pass (backpropagation)
      loss.backward()

      # Update the parameters
      optimizer.step()
   if epoch%10 == 0:
       print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.7600958943367004
epoch:  11  loss:  0.6729137301445007
epoch:  21  loss:  0.7127839922904968
epoch:  31  loss:  0.6517526507377625
epoch:  41  loss:  0.6317514777183533
epoch:  51  loss:  0.5531493425369263
epoch:  61  loss:  0.4423169791698456
epoch:  71  loss:  0.3217153549194336
epoch:  81  loss:  0.33349838852882385
epoch:  91  loss:  0.4703536927700043


### Training Set Log Loss after training loop execution

In [14]:
loss

tensor(0.4893, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [15]:
y_wt_sum = model(X_torch_test.float()) # Equivalent predict_proba / predict
type(y_wt_sum)

torch.Tensor

In [16]:
y_wt_sum[:5]

tensor([[-0.4715],
        [-1.5316],
        [ 2.7476],
        [-1.0996],
        [ 2.4680]], grad_fn=<SliceBackward0>)

In [17]:
y_pred_prob = nn.Sigmoid()(y_wt_sum)
y_pred_prob[:5]

tensor([[0.3843],
        [0.1778],
        [0.9398],
        [0.2498],
        [0.9219]], grad_fn=<SliceBackward0>)

In [18]:
y_pred_prob = y_pred_prob.detach().numpy()
type(y_pred_prob)

numpy.ndarray

In [19]:
y_pred_prob.shape

(40, 1)

In [20]:
y_pred_prob[:5]

array([[0.38425148],
       [0.17776191],
       [0.93977886],
       [0.24981613],
       [0.9218701 ]], dtype=float32)

In [21]:
y_pred_prob = y_pred_prob.reshape(y_test.shape[0],)
y_pred_prob.shape

(40,)

In [22]:
y_pred = np.where(y_pred_prob >= 0.5,1,0)

y_pred[:5]

array([0, 0, 1, 0, 1])

### Test Set Accuracy Score

In [23]:
print(accuracy_score(y_test,y_pred))

0.775


### Test Set Log Loss

In [24]:
log_loss(y_test, y_pred_prob)

0.6148759083684415